In [5]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy
#stolen from class

Below is the OSGeo4W code from the canvas assignment page.  The '64' after each OSGeo4W had to be removed. 

In [6]:
from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"


The next cell copies the shapefile.  The copy is the one I will be editing.  

In [7]:
arcpy.env.overwriteOutput = True
#enables all the magic, I guess
arcpy.management.CopyFeatures(
    "C:\\Users\\seniowl\\458\\Lab1\\saep_bg10\\saep_bg10.shp",
    "C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\saep_bg10_copy2.shp")
#make a copy of the original shapefile in case processing messes with shp beyond repair

<Result 'C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\saep_bg10_copy2.shp'>

The next cell assigns variable names to the copied shapefile and the database file given to us

In [ ]:
wa_tracts_fc = "C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\saep_bg10_copy2.shp" #assigns a variable name to the blockgroup shp
wa_counties_table = "C:\\Users\\seniowl\\458\\Lab1\\WashingtonFIPS.dbf" #assigns a variable name to the county name dbf

The next cell uses the Join Field tool to add the County Name column from the dbf to the blockgroup shapefile based on the three-digit county code that is included in both tables.

In [ ]:
arcpy.JoinField_management(wa_tracts_fc, "COUNTYFP10", wa_counties_table, "FIPSCounty", "CountyName") 
#adds the county name field from the dbf to the blockgroup shp

This cell uses the Dissolve tool to create a new shapefile that has the county names and their total populations.  This is done by combining all of the records with the same county name into one feature.  The only attributes maintained are the county name (the dissolve field) and the summed population (the statistics field).  

In [ ]:
#Dissolve(input, output, field on which the dissolve is based, statisticsfield statistictype)
arcpy.Dissolve_management("C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\saep_bg10_copy2.shp", 
"C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\county_dissolve.shp", 
"CountyNa_1", "POP2017 SUM")
#This tool dissolves the blockgroup layer based on county name while adding all populations
#The previous step worked, 
#but something went wrong with the CountyName field - 
#another field was added called CountyNa_1 which seemed to work
#output is a shp with the 39 counties and their populations


This cell assigns a variable name to the dissolve output shapefile

In [ ]:
county_pops = "C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\county_dissolve.shp"
#assigns a variable name to the new dissolved layer

This cell creates a list of the county names from the database file

In [ ]:
county_list = []
cursor = arcpy.da.SearchCursor(wa_counties_table, ["CountyName"])
for row in cursor:
	county_list.append(row[0])
#creates a list of county names from the dbf

This cell creates the shapefiles for each county, using the list created in the previous cell.  The for loop goes through each county in the list.  The select tool selects records and creates a new shapefile based on the sql statement given as the third parameter in the tool.  Each county name, defined as the variable current_county, is inserted into the name of the shapefile in the second parameter and into the sql statement.  

In [ ]:
#Select_analysis(featureclass, location and name of outputfile, sql statement) 

for current_county in county_list: #
    print current_county #this is here to make sure it is working
    arcpy.Select_analysis(wa_tracts_fc,
                          "C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\"+ current_county + ".shp",
                          "\"CountyNa_1\" = "+ "'" + current_county + "'")
#

This cell makes a list of the total populations based on the dissolve output field created earlier.  Remeber that shp was given the variable name, county_pops.  The population field name of that file became "SUM_POP201", the orignal name of the POP2017 field name from the blockgroup shp, preceded by the statistic type used in the dissolve, and shortened to ten characters.  

The list is then sorted.  It is a numeric field, so it will be sorted numerically.  To sort in descending order, (reverse = True) is added to the function.  listname[0:10] will return the first ten items in the sorted list.  



In [ ]:
#SearchCursor(featureclassname, fieldname)
pop_list = []
cursor = arcpy.da.SearchCursor(county_pops, ["SUM_POP201"]) #county_pops is the dissolved layer, SUM_POP201 is the population field
for row in cursor:
	pop_list.append(row[0]) #creates a list of the populations from the dissolved output
    
pop_list.sort(reverse = True) #sorts the list in descending order
    
print pop_list[0:10] #prints the first ten numbers in the list

The following cleans up that list.  A for loop is used to associate numeric rankings to the populations listed in descending order

In [20]:
#this will make it a bit prettier and print 1-10 ranks
print "Populations of Most Populous Counties in WA:"

i = 1
for populations in pop_list[0:10]:
  	print (str(i) + ": " + str(int(populations)))
        i += 1

Populations of Most Populous Counties in WA:
1: 2153700
2: 859400
3: 789400
4: 499800
5: 471000
6: 276900
7: 264300
8: 253000
9: 216300
10: 193500


I consulted Jonathan on this one.  There were a couple of things that needed to be modified in the code given on the assignment page.  This converts the county shapefiles to geojson files.  I used the same method used for the creation of the county shapefiles.  The county list from the dbf was used in a loop 

In [21]:
for current_county in county_list:
    call(['C:\\OSGeo4W\\bin\\ogr2ogr',
        '-f','GeoJSON','-t_srs','WGS84',
        '-s_srs','EPSG:2927',
        'C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\GeoJSON\\'+ current_county + '.geojson',
        'C:\\Users\\seniowl\\458\\Lab1\\WAcounties\\'+ current_county + '.shp'])

It's hard to say how much time I spent on this assignment.  I would say between reading, looking up tool syntax, writing unsuccessful code, working in class, and producing the final notebook, the entire process took between 17 and 20 hours.  